# Krzysztof Nalepa
## Sprawozdanie z ćwiczenia 10
## Rozwiązywanie układu równań liniowych metodami rozkładu LU
### Podaj zasadę działania metod opartych o dekompozycję LU.

Metoda rozkładu LU polega ona na rozłożeniu macierzy wejściowej na iloczyn macierzy trójkątnej dolnej oraz trójkątnej górnej. Dzięki rozłożeniu macierzy na dwie otrzymujemy dwa równania z macierzami trójkątnymi co znacznie ułatwia rozwiązywanie układu równań.

### Zadanie 1
Zaimplementuj metody rozkładu LU
- Rozkład Crouta
- Rozkład Doolitla
- Rozkład Choleskyego <br>
Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod.
Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9).

#### Warunki
- Rozkład Crouta - Aby zastosować rozkład Crouta musi być spełniony warunek, aby na diagonali macierzy trójkątnej górnej muszą znajdować się wyłącznie 1
- Rozkład Doolitla - Aby zastosować rozkład Doolitla musi być spełniony warunek, aby na diagonali macierzy trójkątnej dolnej muszą znajdować się wyłącznie 1
- Rozkład Cholesky'ego - Aby zastosować rozkład Cholesky'ego wartości na diagonali macierzy trójkątnej górnej oraz na diagonali macierzy trójkątnej dolnej muszą być sobie równe, macierz wejściowa musi być symetryczna oraz dodatnio określona  

In [0]:
from math import sqrt
import numpy as np

def crout_decomposition(A):
    n = len(A)
    L = [[0 for _ in range(n)] for _ in range(n)]
    U = [[0 for _ in range(n)] for _ in range(n)]

    for j in range(n):
        U[j][j] = 1
        for i in range(j, n):
            a = A[i][j]
            for k in range(j):
                a -= L[i][k] * U[k][j]
            L[i][j] = a
        for i in range(j + 1, n):
            tmp = A[j][i]
            for k in range(j):
                tmp -= L[j][k] * U[k][i]
            if int(L[j][j]) == 0:
                raise ZeroDivisionError()
            U[j][i] = tmp / L[j][j]
    return [L, U]

def doolittle_decomposition(A):
    n = len(A)
    L = [[0 for _ in range(n)] for _ in range(n)]
    U = [[0 for _ in range(n)] for _ in range(n)]

    for i in range(n):
        L[i][i] = 1.0
        for j in range(i + 1):
            sum1 = 0
            for k in range(j):
                sum1 += U[k][i] * L[j][k]
            U[j][i] = A[j][i] - sum1
        for j in range(i, n):
            sum2 = 0
            for k in range(i):
                sum2 += U[k][i] * L[j][k]
            L[j][i] = (A[j][i] - sum2) / U[i][i]

    return [L, U]



def cholesky_decomposition(A):
    n = len(A)
    L = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i + 1):
            s = 0
            for k in range(j):
                s += L[i][k] * L[j][k]
            if i == j:
                L[i][j] = math.sqrt(A[i][i] - s)
            else:
                L[i][j] = (1.0 / L[j][j] * (A[i][j] - s))

    return [L, np.array(L).transpose()]

In [69]:
A = [[4, 12, -16],
     [12, 37, -43],
     [-16, -43, 98]]
B = [7, 3, 6]

print("numpy values:\t\t", np.linalg.solve(A, B))


def solve(A, B, fun):
    [L, U] = fun(A)
    z = np.linalg.solve(L, B)
    x = np.linalg.solve(U, z)
    return x

  
print("crout values:\t\t", solve(A, B, crout_decomposition))
print("doolittle values:\t", solve(A, B, doolittle_decomposition))
print("cholesky values:\t", solve(A, B, cholesky))



numpy values:		 [317.52777778 -86.88888889  13.77777778]
crout values:		 [317.52777778 -86.88888889  13.77777778]
doolittle values:	 [317.52777778 -86.88888889  13.77777778]
cholesky values:	 [317.52777778 -86.88888889  13.77777778]


Jak widać każda z metod dekompozycji zadziałała skutecznie, gdyż wynik rozwiązania układu równań w każdym przypadku pokrywa się z oczekiwaną wartością wyliczoną przez numpy

Do przeprowadzenia pomiarów wydajności algorytmów dekompozycji skorzystałem z poniższego skryptu 

In [0]:
def random_sym_matrix(N):
    A = np.random.randint(1, 100, size=(N, N))
    return np.dot(A, A.transpose())


def time_check():
    n = randrange(10, 250)
    A = random_sym_matrix(n).tolist()

    start = time.time()
    crout_decomposition(copy.deepcopy(A))
    stop = time.time()
    crout = stop - start

    start = time.time()
    doolittle_decomposition(copy.deepcopy(A))
    stop = time.time()
    doolittle = stop - start

    start = time.time()
    cholesky_decomposition(copy.deepcopy(A))
    stop = time.time()
    cholesky = stop - start

    return crout, doolittle, cholesky, n


# Main program starts here
if __name__ == '__main__':
    size_list = []
    crout_list = []
    doolittle_list = []
    cholesky_list = []
    for i in range(100):
        crout, doolittle, cholesky, n = time_check()
        crout_list.append(crout)
        doolittle_list.append(doolittle)
        cholesky_list.append(cholesky)
        size_list.append(n)

    f = open("dane.txt", "w+")
    for i in range(100):
        f.write(f"Matrix size: {size_list[i]} \t crout: {crout_list[i]} \t doolittle: {doolittle_list[i]} \t cholesky: {cholesky_list[i]}\n")
    f.close()

    plt.plot(size_list, crout_list, 'bo', label="Crout")
    plt.plot(size_list, doolittle_list, 'ro', label="Doolittle")
    plt.plot(size_list, cholesky_list, 'go', label="Cholesky")
    plt.xlabel("Matrix size")
    plt.ylabel("Time")
    plt.title("Pomiary czasu przeprowadzania dekompozycji")
    plt.legend()
    plt.savefig("wykres1")

Wyniki zamieściłem w pliku pdf 

### Zadanie 3
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.

Biblioteki numpy oraz scipy udostępnia funkcję linlag.solve(), która służy do rozwiązywania układów równań.
W celu porównania wydajności użyłem następującego skryptu

In [0]:

def random_sym_matrix(N):
    A = np.random.randint(1, 100, size=(N, N))
    return np.dot(A, A.transpose())


def random_vector(n):
    return np.random.rand(n)


kromka = 0


def time_check():
    global kromka
    n = randrange(50, 300)
    A = random_sym_matrix(n).tolist()
    B = random_vector(n).tolist()

    print(kromka)
    kromka = kromka + 1
    start = time.time()
    solve(copy.deepcopy(A), copy.deepcopy(B), crout_decomposition)
    stop = time.time()
    crout = stop - start

    start = time.time()
    solve(copy.deepcopy(A), copy.deepcopy(B), doolittle_decomposition)
    stop = time.time()
    doolittle = stop - start

    start = time.time()
    solve(copy.deepcopy(A), copy.deepcopy(B), cholesky_decomposition)
    stop = time.time()
    cholesky = stop - start

    start = time.time()
    np.linalg.solve(copy.deepcopy(A), copy.deepcopy(B))
    stop = time.time()
    numpy = stop - start

    start = time.time()
    linalg.solve(copy.deepcopy(A), copy.deepcopy(B))
    stop = time.time()
    scipy = stop - start

    return crout, doolittle, cholesky, numpy, scipy, n


# Main program starts here
if __name__ == '__main__':
    size_list = []
    crout_list = []
    doolittle_list = []
    cholesky_list = []
    numpy_list = []
    scipy_list = []
    for i in range(200):
        crout, doolittle, cholesky, numpy, scipy, n = time_check()
        crout_list.append(crout)
        doolittle_list.append(doolittle)
        cholesky_list.append(cholesky)
        numpy_list.append(numpy)
        scipy_list.append(scipy)
        size_list.append(n)

    f = open("dane.txt", "w+")
    for i in range(200):
        f.write(
            f"Matrix size: {size_list[i]} \t crout: {crout_list[i]} \t doolittle: {doolittle_list[i]} \t cholesky: {cholesky_list[i]}\n"
            f"numpy: {numpy_list[i]}\t scipy: {scipy_list[i]}\n")
    f.close()

    plt.plot(size_list, crout_list, 'bo', label="Crout")
    plt.plot(size_list, doolittle_list, 'ro', label="Doolittle")
    plt.plot(size_list, cholesky_list, 'go', label="Cholesky")
    plt.plot(size_list, numpy_list, 'yo', label="Numpy")
    plt.plot(size_list, scipy_list, 'co', label="Scipy")
    plt.xlabel("Matrix size")
    plt.ylabel("Time")
    plt.title("Pomiary czasu przeprowadzania rozwiązywania układów równań")
    plt.legend()
    plt.savefig("wykres1")


Wyniki zamieściłem w pliku pdf